# Regularizacija parametara modela 


<center><img src="Images/V9_banner.png" width="700" height="700"/></center>

# Uvod u regularizaciju

Regularizacija je skup tehnika kojima se sprječava **prenaučenost (overfitting)** i poboljšava **generalizacija** modela na neviđene podatke. Tijekom treniranja neuronskih mreža model uči parametre koji minimiziraju funkciju gubitka na skupu za učenje. Međutim, ako je model prekomjerno složen ili ako imamo premalo podataka, mreža može “zapamtiti” trening primjere umjesto da nauči opća pravila. Tada dobivamo vrlo nisku pogrešku na trening skupu, ali visoku pogrešku na test skupu.

Regularizacija uvodi **dodatna ograničenja** ili **penale** na parametre modela kako bi se smanjila mogućnost prekomjernog prilagođavanja šumu u podacima. Time se model usmjerava prema jednostavnijim i stabilnijim rješenjima koja bolje generaliziraju.

Najčešće korištene metode regularizacije uključuju:

- **L2 regularizacija (Ridge)** – kažnjava velika težinska rješenja i potiče glatke modele.  
- **L1 regularizacija (Lasso)** – potiče rijetkost (sparse) parametara i može eliminirati nepotrebne značajke.  
- **Dropout** – nasumično “isključuje” neurone tijekom treniranja, čime sprječava njihovu međusobnu ovisnost.  
- **Early stopping** – prekida treniranje kada se performanse na validacijskom skupu počnu pogoršavati.  
- **Data augmentation** – povećava količinu i raznolikost podataka kako bi model vidio više varijacija.



---

## L1 (Lasso) regularizacija

L1 regularizacija dodaje apsolutne vrijednosti težina u funkciju gubitka:

\begin{align}
\mathcal{L}_{\text{L1}}(\mathbf{w}) = \mathcal{L}_{\text{data}}(\mathbf{w}) + \lambda \sum_{j=1}^{d} |w_j|
\end{align}

L1 potiče **rijetkost (sparsity)** — mnoge težine postaju točno nula.

---

## L2 (Ridge) regularizacija

L2 regularizacija dodaje kvadrate težina:

\begin{align}
\mathcal{L}_{\text{L2}}(\mathbf{w}) = \mathcal{L}_{\text{data}}(\mathbf{w}) + \lambda \sum_{j=1}^{d} w_j^2
\end{align}

L2 potiče **male, glatke vrijednosti težina**, ali rijetko ih postavlja točno na nulu.

---

<font color='green'>


## Primjer

<left><img src="Images/Primjer.png" width="70" height="70"/></left>
</font>

Slijedeća dvije neuronske mreže polučuju isti rezultat -- istu predikciju ($0.2$), ali se bitno razlikuju u težinama $\Theta$:

<center><img src="Images/V9_regularizacija.png" width="700" height="700"/></center>

Neuronska mreža a) ima redom težine $\Theta = [0, 1, 0, 0, 0]$ dok neruonska mreža b) ima težine $\Theta = [-0.03, 0.4, 0.1, 0.2, 0.3]$. Ako se razmilsi, u slučaju neuronske mreže a) ukoliko neuorn sa težinom $1$ iz bilo razloga se ne aktivira, pogreška će automatski biti kriva jer je praktički cijela "odgovornost" postavljena na jedna neuron. S druge strane, neuronska mreža b) u slučaju zatajenja jednog neuorona i dalje ima ostale neurone koji doprinose ukupnoj odluci. Dakle cilj je distribuirati težine kako jedan neuron ne bi imao preveliki utjecaj.

U izrazima za $\text{L1}$ i $\text{L2}$ regularizaciju, s obzirom da obje neuronske mreže daju istu predikciju $0.2$, $\mathcal{L}$ će biti isti, dok $\lambda$ skalira dodatni član jednadžbe koji potiče i uvjetuje oblik težina. Pručimo za obje regularizacije koliko ovi članovi iznose (radi primjera $\lambda = 1$):

Za neuronsku mrežu a), L1: $\lambda \sum_{j=1}^{d} |w_j| = 0+1+0+0+0 = 1$, L2:$\lambda \sum_{j=1}^{d} w_j^2 = 1$

Za neuronsku mrežu b), L1: $\lambda \sum_{j=1}^{d} |w_j| = 0.03 + 0.4 + 0.1 + 0.2 + 0.3 = 1.13$, L2: $\lambda \sum_{j=1}^{d} w_j^2 = 0.0009+0.16+0.01+0.04+0.09=0.3009$ 

Usporedbom uočavamo da L1 norma faovrizira da jedna težina nosi veću odgovornost, dok L2 norma nastoji raspodijeliti odgovornosti na više težina. Sjetite se, cilj je minimizirati funkciju cilja stoga što manji iznost daje element, to je zapravo rješenje bolje.

---

## Dropout regularzacija


Intuicija iza dropouta je vrlo jednostavna: tijekom svake iteracije treniranja **nasumično se izbace (postave na 0)** dio neurona u mreži ([Poveznica](https://jmlr.org/papers/v15/srivastava14a.html)).

To znači:

* svaki **batch** vidi **drugačiju “stanjenu” (thinned) mrežu**,
* mreža se uči ponašati **robustno**, bez oslanjanja na to da su baš svi neuroni prisutni,
* efekt je sličan kao da treniramo **veliki ansambl** mnogo manjih mreža koje **dijele iste parametre**

Na taj način:

* smanjuje se šansu da se neuroni **previše “dogovaraju”** (co-adaptation),  
* poboljšava se **generalizacija** na novim podacima.

-- 
Jedan skriveni sloj ima sljdeće aktivacije

$
\mathbf{h} = f(\mathbf{z})
$  
gdje je $(f)$ neka nelinearna aktivacija (ReLU, tanh, …).

Postupak je sljdeći:

1. **Generira se maska** slučajnih nula i jedinica:
   $
   m_i \sim \text{Bernoulli}(1-p)
   $

2. **Primijeni se maska na aktivacije**:
   $
   \tilde{\mathbf{h}} = \mathbf{m} \odot \mathbf{h}
   $

3. **Nastavlja se propagacija** koristeći $(\tilde{\mathbf{h}})$.

Tijekom **svakog batcha** maska $(\mathbf{m})$ se ponovno nasumično generira.
Svaki neuron je **aktiviran** s vjerojatnošću $(1-p)$, a **gašen** s vjerojatnošću $(p)$.

Bitno je za raspraviti što se događa tijekom inferencije sa dropout slojem.
Postoje dva pristupa:

**1. Standardni dropout (originalni rad):**
* Tijekom treniranja: gasimo dio neurona.
* Tijekom testa: **ne gasimo neurone**, ali skaliramo težine s $(1-p)$.

**2. Inverted dropout (moderna implementacija):**
* Tijekom treniranja:
  $
  \tilde{h}_i = \frac{m_i \cdot h_i}{1-p}
  $
* Tijekom testa: ne radi se ništa posebno, niti skaliramo niti gasimo dropout.

U praksi (PyTorch, TensorFlow) koristi se **inverted dropout**.

---
### Prednosti i nedostaci

**Prednosti:**
- jednostavan za implementirati  
- učinkovita regularizacija  
- radi kao implicitni ensemble  
- dobro se kombinira s L2, augmentacijom, batch normom (ovisno o arhitekturi)

**Nedostaci:**
- može usporiti treniranje (potrebno više epoha)  
- nije uvijek najbolji za moderne arhitekture (npr. veliki Transformeri)  
- potrebno je prilagoditi hiperparametar \(p\)

---

## Augmentacija podataka

Augmentacija podataka je skup tehnika kojima se **umjetno povećava trenirni skup** tako da se postojeće slike ili uzorci transformiraju na način koji **zadržava bitnu semantiku**, ali uvodi dodatnu varijabilnost.

Ključni razlozi zašto se koristi:

* **Smanjuje overfitting** – model vidi više varijacija i ne uči samo “napamet”.
* **Povećava generalizaciju** – bolje performanse na testnim podacima.
* **Pomaže modelu postati robusniji** na rotacije, pomake, šumove, promjene osvjetljenja, perspektive itd.
* **Umjetno povećava dataset** kada je broj uzoraka mali (čest slučaj u medicinskim podacima).
* U nekim slučajevima poboljšava **invarianciju** modela (npr. translacijska invarijancija kod CNN-ova).

Augmentacija se gotovo uvijek koristi **samo tijekom treniranja**, a ne tijekom testiranja.

---
Osnovna ideja augmentacije je vrlo jednostavna: Primijenjuju se nasumične transformacije na ulazne podatke svaki put kad ih model vidi, kako bi isti uzorak svaki put izgledao malo drugačije. Na taj način model nauči:

* ignorirati irelevantne varijacije,
* fokusirati se na bitne karakteristike,  
* postati robusniji na realne uvjete izvan trenirnog skupa.

---

Najčešće korištene augmentacije: 

#### 1. Geometrijske transformacije

##### **Random rotacije**
* Slike se rotiraju za neki slučajni kut (npr. ±15°).
* Pomaže kod orijentacijskih varijacija.

##### **Random translacije (pomaci)**
* Slika se pomakne po x ili y osi.
* Pomaže kod translacijske invariancije.

##### **Random skaliranje (zoom in/out)**
* Slika se slučajno približi ili udalji.
* Pomaže CNN-u da se ne oslanja na apsolutnu veličinu objekta.

##### **Random horizontal flip**
* Najjednostavnija augmentacija.
* Koristi se za prirodne slike; oprez kod medicinskih slika.

##### **Random vertical flip**
* Također korisno, ali pazi kod anatomije (npr. nije validno za CT glave).

##### **Random cropping**
* Isječe se nasumičan dio slike.
* Uči model robusnosti na pozicioniranje i scale.

##### **Random perspective / affine transformacije**
* Blage promjene perspektive.
* Korisno u objektnoj detekciji i klasifikaciji.

#### 2. Fotometrijske transformacije

##### **Promjene osvjetljenja**
* Brightness, contrast, gamma.
* Simulira različite uvjete snimanja.

##### **Promjene zasićenja i nijanse**
* Koristi se kod RGB slika, manje u medicinskim datasetima.

##### **Dodavanje šuma**
* Gaussian noise.
* Model postaje robusniji na “realne” smetnje u podacima.

##### **Gaussian blur**
* Koristi se umjereno; model se uči nositi s mutnim podacima.

##### **Color jitter**
* Kombinira brightness/contrast/saturation/hue promjene.

#### 3. Prostorne i strukturalne augmentacije

##### **Cutout / Random Erasing**
* Nasumično se “izbriše” mali kvadrat slike.
* Sprječava model da se oslanja na jako male lokalne feature-e.

##### **Mixup**
* Kombinira dvije slike i labele:  
  $
  \tilde{x} = \lambda x_1 + (1-\lambda) x_2
  \]
  \[
  \tilde{y} = \lambda y_1 + (1-\lambda) y_2
  $
* Povećava linearizira granice odlučivanja.

##### **CutMix**
* Izreže se patch iz jedne slike i zalijepi na drugu.
* Kombinira prednosti cutout-a i mixup-a.

##### **Random elastic deformation**
* Koristi se često u medicinskoj obradi slike (npr. MRI/CT).
* Simulira anatomske deformacije.

---

#### 4. Specifične augmentacije za medicinske slike

* **Histogram equalization / CLAHE**  
  poboljšava kontrast kod slabo osvijetljenih skenova.

* **Random intensity shifts**  
  posebno korisno kod CT-a (promjene HU vrijednosti).

* **Random gamma correction**  
  prilagodba dinamike osvjetljenja.

* **Elastic deformation**  
  simulira anatomske varijacije.

* **Spatial dropout / channel dropout**  
  posebno za 3D CNN arhitekture.

---

### Kada augmentacija može biti loša?

* Ako stvara **nerealistične primjere** (npr. flipping kod anatomije glave).  
* Ako transformacije naruše **semantiku** (npr. prevelika rotacija kod X-ray slika može promijeniti smjer anatomije).  
* Ako se previše agresivno augmentira → model "nauči" na nenormalnim primjerima.

Pravila:

* Geometrijske augmentacije: koristiti umjereno.  
* Fotometrijske augmentacije: koristiti ovisno o domenskom znanju.  
* U medicini: uvijek se zapitati **“može li ovakva slika postojati u stvarnom svijetu?”**.

---

## Transfer Learning

Treniranje dubokih neuronskih mreža od nule zahtijeva:

* **velike količine podataka**
* **veliku računalnu snagu**
* **dugo vrijeme treniranja**

U praksi vrlo često **nemamo dovoljno podataka**, osobito u medicini, malim domenama ili specifičnim klasifikacijskim zadacima.  
Zbog toga koristimo **transfer learning**, tehniku u kojoj “posuđujemo” znanje naučeno na velikim skupovima podataka.

**Transfer learning** je pristup u kojem se model prvo trenira (ili uzmemo već istreniran model) na **velikom, općem datasetu**, a zatim ga **prilagođavamo novom, manjem i specifičnom problemu**. Tipičan primjer jest neuoronska mreža trenirana na ImageNet-u (1.2M slika, 1000 klasa) nauči univerzalne značajke (edge, texture, shapes). Te značajke mogu se **prijenijeti** na medicinske slike, prijenos domene, male datasete, i sl.

### Prednosti transfer learninga

* **brže treniranje** (model kreće od “pametnog” začetnog stanja)
* **bolja generalizacija** na malim datasetima
* **manji rizik od overfittinga**
* često omogućuje **state-of-the-art rezultate** uz minimalno treniranje
* u medicini često **nužan** zbog malih datasetova


### Tri glavna načina transfer learninga

##### 1. **Feature Extraction (zamrzavanje cijele mreže)**
- Zamrznemo sve slojeve osim zadnjeg.
- Model koristi već naučene feature mapove.
- Brzo, stabilno; dobro za **male datasete**.

##### 2. **Fine-Tuning (otključamo samo dio mreže)**
- Prvo zamrznemo sve slojeve, treniramo samo zadnji sloj.
- Zatim otključamo više slojeva i treniramo s malim LR.
- Dobro kada imamo **srednje velik dataset**.

##### 3. **Full Fine-Tuning**
- Svi slojevi se treniraju.
- Koristi se kada postoji **dovoljno podataka** ili kada je domena slična (npr. CT → CT).

---
Transfer larning daje benefite jer veliki modeli uče: rubove, teksture, oblike, kompozicijske strukture...
Ove karakteristike su **općenite** i korisne za mnoge zadatke. Npr. CNN koji nauči prepoznati rubove na ImageNet-u, može te iste značajke koristiti i na medicinskim X-ray ili CT slikama. Time se izbjega treniranje od nule i koristi se “već naučena vizualnu abecedu”.

Transfer learning s druge strane neće pomoći u slučaju kada:

* Domene su previše različite:  
  npr. ImageNet (RGB životinje i objekti) → MRI volumeni (grayscale, 3D).  
  Ipak, čak i tada često daje bolje rezultate nego random inicijalizacija.

* Posjedujemo **iznimno veliki skup podataka** -- bolje je trenirati od nule.

* Ako su ulazni podaci vrlo specifični (npr. multispektralne satelitske slike), generalne značajke možda nisu idealne.


---

<font color='red'>


## Zadatak

<left><img src="Images/Zadatak.png" width="70" height="70"/></left>
</font>

Idući programski kod omogućava istraživanje većine navedenih koncepata. Glavne značajke skupa podataka su:

* Rad na MNIST skupu poadatak,
* Definiranje dubinu i širinu mreže,
* Tip i jačinu ($\lambda$) L1/L2 regularizacije i dropout regularizacije,
* Inspekcija norme gradijenata (nestajanje/eksplodiranje) kroz epohe,
* Crtatnje box-plot distribucije aktivacija i gradijenata po izabranom sloju i epohama,
* Pratiti macro F1 na validacijskom skupu kroz epohe te jednu završnu točku macro F1 na test skupu.

Vaš je zadatak proučiti utjecaj navedenih regularizacije i hiperparametara mreže na konačne rezultate predikcija i same gradijente. Usporedite gradijente i preciznosti modela za modele u kojima se mijenjaju samo navedeni parametri (ista topologija, stopa učenja itd.):
* L1 sa $\lambda = 0.001$,
* L2 sa $\lambda 0.01$,
* dropout sa 0.15,
* Neuronska mreža bez regularzicaije




In [2]:
# Autoreload
%load_ext autoreload
%autoreload 2
    
# Učitavanje programskih knjižnica
from Skripte.Vjezba9.widget import deep_mlp_training_widget

ui = deep_mlp_training_widget()
ui


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
